In [1]:
from keras.models import Sequential
from keras.layers import Dense

# import librart
import sklearn
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
from scipy.sparse import csr_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import feature_select
import feture_process
from scipy.sparse import load_npz
from scipy.sparse import load_npz
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [2]:
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")

One hot code Processing

In [3]:
# process Prublisher and Language
train_df["Language"], test_df["Language"] = feture_process.docclass_preprocess(train_df["Language"],test_df["Language"],10)
train_df["Publisher"], test_df["Publisher"] = feture_process.docclass_preprocess(train_df["Publisher"],test_df["Publisher"],200)

In [4]:
# use oneHotCode for Publisher (sklearn)
publisher_train_hot = feture_process.process_OneHotEncoder(train_df,"Publisher")
publisher_test_hot = feture_process.process_OneHotEncoder(test_df,"Publisher")
train_df = pd.concat([train_df, publisher_train_hot], axis=1)
test_df = pd.concat([test_df, publisher_test_hot], axis=1)

In [5]:
# use oneHotCode for Language (sklearn)
language_train_hot = feture_process.process_OneHotEncoder(train_df,"Language")
language_test_hot = feture_process.process_OneHotEncoder(test_df,"Language")
train_df = pd.concat([train_df, language_train_hot], axis=1)
test_df = pd.concat([test_df, language_test_hot], axis=1)

In [6]:
# use oneHotCode for label (sklearn)
label_train_hot = feture_process.process_OneHotEncoder(train_df,"rating_label")
train_df = pd.concat([train_df, label_train_hot], axis=1)

Countvectorizer processing

In [7]:
# train_name_countvectorizer
train_name_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_name_countvectorizer.pkl", "rb"))
train_name_dic = train_name_countvectorizer.vocabulary_

# train_authors_countvectorizer
train_authors_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_authors_countvectorizer.pkl", "rb"))
train_authors_dic = train_authors_countvectorizer.vocabulary_

# train_desc_countvectorizer
train_desc_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_desc_countvectorizer.pkl", "rb"))
train_desc__dic = train_desc_countvectorizer.vocabulary_

# process vector features
train_name_features = train_name_countvectorizer.transform(train_df['Name'])
train_authors_features = train_authors_countvectorizer.transform(train_df['Authors'])
train_desc_features = train_desc_countvectorizer.transform(train_df['Description'])
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language','rating_label_3.0', 'rating_label_4.0', 'rating_label_5.0', 'rating_label'])

In [8]:
# new sparse features
sparse_features = hstack([train_name_features, train_authors_features, train_desc_features])
# new train features
dense_features = csr_matrix(other_features_df.values)
train_features = hstack([sparse_features, dense_features])


# process test features
test_name_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_name_vec.npz')
test_authors_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_authors_vec.npz')
test_desc_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_desc_vec.npz')
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])

test_sparse_features = hstack([test_name_features, test_authors_features, test_desc_features])
new_dense_features = csr_matrix(test_other_features_df.values)
test_features = hstack([test_sparse_features, new_dense_features])

doc2vec processing

In [ ]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# process vector features
train_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
train_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
train_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language', 'rating_label', 'rating_label_3.0', 'rating_label_4.0', 'rating_label_5.0'])
train_features = pd.concat([train_name_features, train_authors_features, train_desc_features, other_features_df], axis=1)

In [ ]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# process test features
test_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
test_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])
test_features = pd.concat([test_name_features, test_authors_features, test_desc_features, test_other_features_df], axis=1)


Feature selection

In [ ]:
selected_features,selected_features_test = train_features,test_features

In [ ]:
selected_features,selected_features_test = feature_select.MI(train_df,train_features,test_features,12000)

In [34]:
selected_features,selected_features_test = feature_select.chi_square(train_df,train_features,test_features,12000)

In [ ]:
selected_features = load_npz("selected_features_label_one_hot_12000.npz")
selected_features_test = load_npz("selected_features_test_label_one_hot_12000.npz")

In [35]:
from keras.regularizers import l1, l2


input_dim = selected_features.shape[1] 
num_classes = label_train_hot.shape[1]

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(input_dim,), kernel_regularizer=l2(0.0001)))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [36]:
X_train, X_val, y_train, y_val = train_test_split(selected_features, label_train_hot, test_size=0.3, random_state=66)

In [37]:
X_train = X_train.toarray()
X_val = X_val.toarray()

In [38]:
from sklearn.utils import class_weight
unique_classes = np.unique(y_train)
sample_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)

In [40]:
mlp = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), sample_weight=sample_weights)

Epoch 1/50
505/505 [==============================] - 6s 12ms/step - loss: 1.1608 - accuracy: 0.1887 - val_loss: 1.2803 - val_accuracy: 0.2727
Epoch 2/50
505/505 [==============================] - 6s 11ms/step - loss: 0.8854 - accuracy: 0.2605 - val_loss: 1.7428 - val_accuracy: 0.1828
Epoch 3/50
505/505 [==============================] - 6s 11ms/step - loss: 0.7368 - accuracy: 0.3344 - val_loss: 1.5214 - val_accuracy: 0.2743
Epoch 4/50
505/505 [==============================] - 5s 11ms/step - loss: 0.5935 - accuracy: 0.4431 - val_loss: 1.0175 - val_accuracy: 0.5244
Epoch 5/50
505/505 [==============================] - 5s 11ms/step - loss: 0.6080 - accuracy: 0.4806 - val_loss: 1.4510 - val_accuracy: 0.3154
Epoch 6/50
505/505 [==============================] - 5s 11ms/step - loss: 0.5093 - accuracy: 0.5258 - val_loss: 1.1732 - val_accuracy: 0.4188
Epoch 7/50
505/505 [==============================] - 5s 11ms/step - loss: 0.4697 - accuracy: 0.5773 - val_loss: 1.5309 - val_accuracy: 0.3094

In [ ]:
prediction_prob = model.predict(selected_features_test)

In [ ]:
# get predictions
prediction_index = np.argmax(prediction_prob, axis=1)
ratings = np.array([3.0, 4.0, 5.0])
prediction = ratings[prediction_index]

In [ ]:
output_df = pd.DataFrame({'rating_label': prediction})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')